In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance

In [ ]:
directory = '../input/state-farm-distracted-driver-detection/imgs/train'
test_directory = '../input/state-farm-distracted-driver-detection/imgs/test'
random_test = '../input/driver/'
classes = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']

In [ ]:
img_size1 = 240
img_size2 = 240

In [ ]:
training_data = []
i = 0
def create_training_data():
    for category in classes:
        path = os.path.join(directory,category)
        class_num = classes.index(category)
        
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
            new_img = cv2.resize(img_array,(img_size2,img_size1))
            training_data.append([
                new_img,class_num])


In [ ]:
testing_data = []
i = 0
def create_testing_data():        
    for img in os.listdir(test_directory):
        img_array = cv2.imread(os.path.join(test_directory,img),cv2.IMREAD_GRAYSCALE)
        new_img = cv2.resize(img_array,(img_size2,img_size1))
        testing_data.append([img,
            new_img])

In [ ]:
import cv2

In [ ]:
create_training_data()

In [ ]:
create_testing_data()

In [ ]:
import random

In [ ]:
random.shuffle(training_data)

In [ ]:
x = []
y = []

In [ ]:
for features, label in training_data:
    x.append(features)
    y.append(label)

In [ ]:
from sklearn.model_selection import train_test_split
X = np.array(x).reshape(-1,img_size2,img_size1,1)
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=50)

In [ ]:
Y_train = tf.keras.utils.to_categorical(y_train,num_classes=10)
Y_test = tf.keras.utils.to_categorical(y_test,num_classes=10)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(240,240,1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.BatchNormalization(axis = 3))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.BatchNormalization(axis = 3))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.BatchNormalization(axis = 3))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units = 512,activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units = 128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=5)]

In [ ]:
batch_size = 50
n_epochs = 10

In [ ]:
results = model.fit(x_train,Y_train,batch_size=batch_size,epochs=n_epochs,verbose=1,validation_data=(x_test,Y_test),callbacks=callbacks)

In [ ]:
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
preds = model.predict(np.array(testing_data[0][1]).reshape(-1,img_size2,img_size1,1))

In [ ]:
model.save_weights('./driverdistraction_lr_weights.h5', overwrite=True)

In [ ]:
model.save('./driverdistraction_lr_weights.h5')

In [ ]:
loaded_model = tf.keras.models.load_model('./driverdistraction_lr_weights.h5')

In [ ]:
test_data = np.array(testing_data[3000][1]).reshape(-1,img_size2,img_size1,1)

In [ ]:
preds = loaded_model.predict(test_data)

In [ ]:
print('Predicted: {}'.format(np.argmax(preds)))

In [ ]:
testing_data[1]

In [ ]:
submission=None
submission=pd.DataFrame(columns=['img','c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])

In [ ]:
pd.read_csv('../input/state-farm-distracted-driver-detection/sample_submission.csv')

In [ ]:
for  n,i in testing_data[:10000]:
    pred=model.predict(i.reshape(-1,img_size2,img_size1,1))
    one=(np.argmax(pred))
    row=[n,0,0,0,0,0,0,0,0,0,0]
    row[one+1]=1
    submission.loc[len(submission.index)] = row

submission
    

In [ ]:
submission.to_csv('./submission.csv')

In [ ]:
for  n,i in testing_data[10000:20000]:
    pred=model.predict(i.reshape(-1,img_size2,img_size1,1))
    one=(np.argmax(pred))
    row=[n,0,0,0,0,0,0,0,0,0,0]
    row[one+1]=1
    submission.loc[len(submission.index)] = row

submission

In [ ]:
for  n,i in testing_data[20000:30000]:
    pred=model.predict(i.reshape(-1,img_size2,img_size1,1))
    one=(np.argmax(pred))
    row=[n,0,0,0,0,0,0,0,0,0,0]
    row[one+1]=1
    submission.loc[len(submission.index)] = row

submission

In [ ]:
for  n,i in testing_data[30000:40000]:
    pred=model.predict(i.reshape(-1,img_size2,img_size1,1))
    one=(np.argmax(pred))
    row=[n,0,0,0,0,0,0,0,0,0,0]
    row[one+1]=1
    submission.loc[len(submission.index)] = row

submission

In [ ]:
for  n,i in testing_data[40000:50000]:
    pred=model.predict(i.reshape(-1,img_size2,img_size1,1))
    one=(np.argmax(pred))
    row=[n,0,0,0,0,0,0,0,0,0,0]
    row[one+1]=1
    submission.loc[len(submission.index)] = row

submission

In [ ]:
for  n,i in testing_data[50000:60000]:
    pred=model.predict(i.reshape(-1,img_size2,img_size1,1))
    one=(np.argmax(pred))
    row=[n,0,0,0,0,0,0,0,0,0,0]
    row[one+1]=1
    submission.loc[len(submission.index)] = row

submission

In [ ]:
for  n,i in testing_data[60000:70000]:
    pred=model.predict(i.reshape(-1,img_size2,img_size1,1))
    one=(np.argmax(pred))
    row=[n,0,0,0,0,0,0,0,0,0,0]
    row[one+1]=1
    submission.loc[len(submission.index)] = row

submission

In [ ]:
for  n,i in testing_data[70000:]:
    pred=model.predict(i.reshape(-1,img_size2,img_size1,1))
    one=(np.argmax(pred))
    row=[n,0,0,0,0,0,0,0,0,0,0]
    row[one+1]=1
    submission.loc[len(submission.index)] = row

submission

In [ ]:
submission.sort_values(by=['img']).to_csv('final2.csv',index=False)